In [ ]:
import numpy as np
# from IPython import display
import sys
sys.path.append("/home/gareth/XLB")
import matplotlib.pylab as plt
from src.models import BGKSim, KBCSim
from src.lattice import LatticeD3Q19, LatticeD3Q27
from src.boundary_conditions import *
import numpy as np
from src.utils import *
from jax import config
import os
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8'
import jax
import scipy

In [ ]:
%ls ~

XLB/          color_testing/  readme.txt   taskwarrior_data/
__pycache__/  gareth/         taskd_data/


In [ ]:
class Car(KBCSim):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def voxelize_stl(self, stl_filename, length_lbm_unit):
        mesh = trimesh.load_mesh(stl_filename, process=False)
        length_phys_unit = mesh.extents.max()
        pitch = length_phys_unit/length_lbm_unit
        mesh_voxelized = mesh.voxelized(pitch=pitch)
        mesh_matrix = mesh_voxelized.matrix
        return mesh_matrix, pitch

    def set_boundary_conditions(self):
        print('Voxelizing mesh...')
        time_start = time()
        stl_filename = 'stl-files/DrivAer-Notchback.stl'
        car_length_lbm_unit = self.nx / 4
        car_voxelized, pitch = voxelize_stl(stl_filename, car_length_lbm_unit)
        car_matrix = car_voxelized.matrix
        print('Voxelization time for pitch={}: {} seconds'.format(pitch, time() - time_start))
        print("Car matrix shape: ", car_matrix.shape)

        self.car_area = np.prod(car_matrix.shape[1:])
        tx, ty, tz = np.array([nx, ny, nz]) - car_matrix.shape
        shift = [tx//4, ty//2, 0]
        car_indices = np.argwhere(car_matrix) + shift
        self.BCs.append(BounceBackHalfway(tuple(car_indices.T), self.gridInfo, self.precisionPolicy))

        wall = np.concatenate((self.boundingBoxIndices['bottom'], self.boundingBoxIndices['top'],
                               self.boundingBoxIndices['front'], self.boundingBoxIndices['back']))
        self.BCs.append(BounceBack(tuple(wall.T), self.gridInfo, self.precisionPolicy))

        doNothing = self.boundingBoxIndices['right']
        self.BCs.append(DoNothing(tuple(doNothing.T), self.gridInfo, self.precisionPolicy))
        self.BCs[-1].implementationStep = 'PostCollision'
        # rho_outlet = np.ones(doNothing.shape[0], dtype=self.precisionPolicy.compute_dtype)
        # self.BCs.append(ZouHe(tuple(doNothing.T),
        #                                          self.gridInfo,
        #                                          self.precisionPolicy,
        #                                          'pressure', rho_outlet))

        inlet = self.boundingBoxIndices['left']
        rho_inlet = np.ones((inlet.shape[0], 1), dtype=self.precisionPolicy.compute_dtype)
        vel_inlet = np.zeros(inlet.shape, dtype=self.precisionPolicy.compute_dtype)

        vel_inlet[:, 0] = prescribed_vel
        self.BCs.append(EquilibriumBC(tuple(inlet.T), self.gridInfo, self.precisionPolicy, rho_inlet, vel_inlet))
        # self.BCs.append(ZouHe(tuple(inlet.T),
        #                                          self.gridInfo,
        #                                          self.precisionPolicy,
        #                                          'velocity', vel_inlet))

In [ ]:

precision = 'f32/f32'
lattice = LatticeD3Q27(precision)

nx = 601
ny = 351
nz = 251

Re = 50000.0
prescribed_vel = 0.05
clength = nx - 1

visc = prescribed_vel * clength / Re
omega = 1.0 / (3. * visc + 0.5)

os.system('rm -rf ./*.vtk && rm -rf ./*.png')

kwargs = {
    'lattice': lattice,
    'omega': omega,
    'nx': nx,
    'ny': ny,
    'nz': nz,
    'precision': precision,
    'io_rate': 100,
    'print_info_rate': 100,
    'return_fpost': True  # Need to retain fpost-collision for computation of lift and drag
}
sim = Car(**kwargs)
sim.run(200000)

#NB if you get an error here, it is because cuda needs updateing
# See https://jax.readthedocs.io/en/latest/installation.html#nvidia-gpu

**** Simulation Parameters for Car ****
            Parameter | Value
--------------------------------------------------
                Omega | 1.9928258270227182
     Grid Points in X | 601
     Grid Points in Y | 351
     Grid Points in Z | 251
       Dimensionality | 3
     Precision Policy | f32/f32
         Lattice Type | D3Q27
      Checkpoint Rate | 0
 Checkpoint Directory | ./checkpoints
  Downsampling Factor | 1
      Print Info Rate | 100
             I/O Rate | 100
        Compute MLUPS | False
   Restore Checkpoint | False
              Backend | cpu
    Number of Devices | 1
Voxelizing mesh...


ValueError: string is not a file: stl-files/DrivAer-Notchback.stl